In [1]:
import torch
import pandas as pd
import numpy as np
import csv
from hanspell import spell_checker
from collections import Counter
# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran
# 워드클라우드
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter

In [4]:
kurly_data = pd.read_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\kurly_data_preprocessed.csv", index_col=0)

In [5]:
kurly_data.head(2)

,ProductCode,MemberGrade,MemberName,ProductName,ReviewText,ReviewDate,ReviewLike,token_okt
0,5136210,일반,박**,[바로생활] 아카시아꿀 스틱 10포,작은 박스로 2개 (10개씩 들어있어 20개네요 ) 주문했습니다. 의심이 많은 타...,2023.06.23,0,"[('작은', 'Adjective'), ('박스', 'Noun'), ('로', 'J..."
1,5136210,프렌즈,강**,[바로생활] 아카시아꿀 스틱 10포,물에 타먹으니까 되게 다네요,2023.08.15,0,"[('물', 'Noun'), ('에', 'Josa'), ('타', 'Noun'), ..."


### 감성분석

In [ ]:
# import library
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# load model
tokenizer = AutoTokenizer.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

In [ ]:
review_list = list(kurly_data['ReviewText'])

In [10]:
# 감정분석 결과 및 점수
pred_label = []
pred_score = []
for rev in review_list:
    label_temp = sentiment_classifier(rev)[0]['label']
    score_temp = sentiment_classifier(rev)[0]['score']
    pred_label.append(label_temp)
    pred_score.append(score_temp)

In [13]:
kurly_data['sentiment_label'] = pred_label
kurly_data['sentiment_score'] = pred_score

In [16]:
kurly_data['sentiment_label'].value_counts()

sentiment_label
1    10085
0      569
Name: count, dtype: int64

In [46]:
# 리뷰 출력시 ... 없이 전부 나오게 설정
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)

In [34]:
kurly_data['ReviewText'][0]

'작은 박스로 2개 (10개씩 들어있어 20개네요 ) 주문했습니다.  의심이 많은 타입이라 뭐든 한 번에 주문한 적이 없고 ,,  꼭 먹어보고 써보고 나서야 안심하는 타입입니다 ㅋ    요즘은.. 아카시아가 많이 없어 ..  아카시아 향 맡아보기도 정말 힘든데.. 아카시아 꿀이라니  ^^  너무 반갑고 예뻐 주문해 봤습니다.   일단 너무 작고 예쁘네요.  더운 여름에 밖에서 운동하는 딸을 위해 건강한 간식으로  챙겨 주려고 합니다.   예쁜 포장이라 예민한 아이 마음에도 쏙 드는 듯합니다.   배송도 빠르고 좋은 제품 좋은 가격에 잘 산 걸 같아  기분 좋네요.^^'

품사 선택

In [ ]:
# 튜플 리스트 받아서 명사/동사/형용사만 추출
# word_class = ['Noun', 'Verb', 'Adjective']
def token_select(token_tuple_list, word_class):
    selected_token = []
    for token_review in token_tuple_list:
        if token_review[1] in word_class:
            selected_token.append(token_review[0])
    return selected_token

In [21]:
import sys
import google.protobuf.text_format as tf
from bareunpy import Tokenizer

# If you have your own localhost bareun.
my_tokenizer = Tokenizer('koba-RCCJZSI-RI7UFUI-TIRFYWI-LXMRWDY', 'localhost')

In [35]:
tokenized = my_tokenizer.tokenize_list([kurly_data['ReviewText'][0]])

In [ ]:
# get protobuf message.
m = tokenized.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
print(tf.MessageToString(m, as_utf8=True))
print(f'length of sentences is {len(m.sentences)}')
## output : 2
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of segments of first token in sentences[0] is {len(m.sentences[0].tokens[0].segments)}')
print(f'tagged of first token in sentences[0] is {m.sentences[0].tokens[0].tagged}')
print(f'first segment of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0]}')
print(f'hint of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0].hint}')

In [ ]:

# or if you have your own bareun which is running on 10.8.3.211:15656.
#my_tokenizer = Tagger('koba-RCCJZSI-RI7UFUI-TIRFYWI-LXMRWDY', '10.8.3.211', 15656)


# print results. 
tokenized = my_tokenizer.tokenize_list(["안녕하세요.", "반가워요!"])

# get protobuf message.
m = tokenized.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
print(tf.MessageToString(m, as_utf8=True))
print(f'length of sentences is {len(m.sentences)}')
## output : 2
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of segments of first token in sentences[0] is {len(m.sentences[0].tokens[0].segments)}')
print(f'tagged of first token in sentences[0] is {m.sentences[0].tokens[0].tagged}')
print(f'first segment of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0]}')
print(f'hint of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0].hint}')

## Advanced usage.
for sent in m.sentences:
    for token in sent.tokens:
        for m in token.segments:
            print(f'{m.text.content}/{m.hint}')

# get json object
jo = tokenized.as_json()
print(jo)

# get tuple of segments
ss = tokenized.segments()
print(ss)
ns = tokenized.nouns()
print(ns)
vs = tokenized.verbs()
print(vs)
# postpositions: 조사
ps = tokenized.postpositions()
print(ps)
# Adverbs, 부사
ass = tokenized.adverbs()
print(ass)
ss = tokenized.symbols()
print(ss)


In [36]:
tokenized = my_tokenizer.tokenize_list([kurly_data['ReviewText'][0]])

In [ ]:
kurly_data['token_bareun_noun'] = kurly_data['ReviewText'].apply(lambda x:my_tokenizer)

In [52]:
my_tokenizer.tokenize(kurly_data['ReviewText'][0]).verbs()

['작',
 '들',
 '있',
 '이',
 '하',
 '많',
 '이',
 '하',
 '없',
 '먹',
 '보',
 '쓰',
 '보',
 '나',
 '하',
 '이',
 '없',
 '맡아보',
 '힘들',
 '이',
 '반갑',
 '예쁘',
 '하',
 '보',
 '작',
 '예쁘',
 '덥',
 '하',
 '위하',
 '하',
 '챙기',
 '주',
 '하',
 '예쁘',
 '이',
 '예민하',
 '들',
 '하',
 '빠르',
 '좋',
 '좋',
 '살',
 '같',
 '좋']

In [37]:
tokenized.as_json()['sentences']

[{'text': {'content': '작은 박스로 2개 (10개씩 들어있어 20개네요 ) 주문했습니다.  의심이 많은 타입이라 뭐든 한 번에 주문한 적이 없고 ,,  꼭 먹어보고 써보고 나서야 안심하는 타입입니다 ㅋ    요즘은.. 아카시아가 많이 없어 ..  아카시아 향 맡아보기도 정말 힘든데.. 아카시아 꿀이라니  ^^  너무 반갑고 예뻐 주문해 봤습니다.   일단 너무 작고 예쁘네요.  더운 여름에 밖에서 운동하는 딸을 위해 건강한 간식으로  챙겨 주려고 합니다.   예쁜 포장이라 예민한 아이 마음에도 쏙 드는 듯합니다.   배송도 빠르고 좋은 제품 좋은 가격에 잘 산 걸 같아  기분 좋네요.^^',
   'length': 319,
   'beginOffset': 0},
  'tokens': [{'text': {'content': '작은', 'length': 2, 'beginOffset': 0},
    'segments': [{'text': {'content': '작', 'length': 1, 'beginOffset': 0},
      'hint': 'V'},
     {'text': {'content': '은', 'beginOffset': 1, 'length': 1}, 'hint': 'E'}],
    'tagged': '작/V+은/E'},
   {'text': {'content': '박스로', 'beginOffset': 3, 'length': 3},
    'segments': [{'text': {'content': '박스', 'beginOffset': 3, 'length': 2},
      'hint': 'N'},
     {'text': {'content': '로', 'beginOffset': 5, 'length': 1}, 'hint': 'J'}],
    'tagged': '박스/N+로/J'},
   {'text': {'content': '2개', 'beginOffset': 7, 'length': 2},
    'segments': [{

In [38]:
tokenized.segments()

['작',
 '은',
 '박스',
 '로',
 '2',
 '개',
 '(',
 '10',
 '개',
 '씩',
 '들',
 '어',
 '있',
 '어',
 '20',
 '개',
 '이',
 '네',
 '요',
 ')',
 '주문',
 '하',
 '았',
 '습니다',
 '.',
 '의심',
 '이',
 '많',
 '은',
 '타입',
 '이',
 '라',
 '뭐',
 '든',
 '한',
 '번',
 '에',
 '주문',
 '하',
 'ㄴ',
 '적',
 '이',
 '없',
 '고',
 ',',
 ',',
 '꼭',
 '먹',
 '어',
 '보',
 '고',
 '쓰',
 '어',
 '보',
 '고',
 '나',
 '아서',
 '야',
 '안심',
 '하',
 '는',
 '타입',
 '이',
 'ㅂ니다',
 'ㅋ',
 '요즘',
 '은',
 '.',
 '.',
 '아카시아',
 '가',
 '많이',
 '없',
 '어',
 '.',
 '.',
 '아카시아',
 '향',
 '맡아보',
 '기',
 '도',
 '정말',
 '힘들',
 'ㄴ데',
 '.',
 '.',
 '아카시아',
 '꿀',
 '이',
 '라니',
 '^',
 '^',
 '너무',
 '반갑',
 '고',
 '예쁘',
 '어',
 '주문',
 '하',
 '아',
 '보',
 '았',
 '습니다',
 '.',
 '일단',
 '너무',
 '작',
 '고',
 '예쁘',
 '네',
 '요',
 '.',
 '덥',
 'ㄴ',
 '여름',
 '에',
 '밖',
 '에서',
 '운동',
 '하',
 '는',
 '딸',
 '을',
 '위하',
 '아',
 '건강',
 '하',
 'ㄴ',
 '간식',
 '으로',
 '챙기',
 '어',
 '주',
 '려고',
 '하',
 'ㅂ니다',
 '.',
 '예쁘',
 'ㄴ',
 '포장',
 '이',
 '라',
 '예민하',
 'ㄴ',
 '아이',
 '마음',
 '에',
 '도',
 '쏙',
 '들',
 '는',
 '듯',
 '하',
 'ㅂ니다',
 '.',
 '배송',
 '도'

In [39]:
tokenized.nouns()

['박스',
 '2',
 '개',
 '10',
 '개',
 '씩',
 '20',
 '개',
 '주문',
 '의심',
 '타입',
 '뭐',
 '번',
 '주문',
 '적',
 '안심',
 '타입',
 'ㅋ',
 '요즘',
 '아카시아',
 '아카시아',
 '향',
 '아카시아',
 '꿀',
 '주문',
 '여름',
 '밖',
 '운동',
 '딸',
 '건강',
 '간식',
 '포장',
 '아이',
 '마음',
 '듯',
 '배송',
 '제품',
 '가격',
 '거',
 '기분']

In [40]:
tokenized.verbs()

['작',
 '들',
 '있',
 '이',
 '하',
 '많',
 '이',
 '하',
 '없',
 '먹',
 '보',
 '쓰',
 '보',
 '나',
 '하',
 '이',
 '없',
 '맡아보',
 '힘들',
 '이',
 '반갑',
 '예쁘',
 '하',
 '보',
 '작',
 '예쁘',
 '덥',
 '하',
 '위하',
 '하',
 '챙기',
 '주',
 '하',
 '예쁘',
 '이',
 '예민하',
 '들',
 '하',
 '빠르',
 '좋',
 '좋',
 '살',
 '같',
 '좋']

In [42]:
from transformers import ElectraTokenizerFast, ElectraModel, TFElectraModel
tokenizer_electra = ElectraTokenizerFast.from_pretrained("kykim/electra-kor-base")

In [47]:
tokenizer_electra.tokenize(kurly_data['ReviewText'][0])

['작은',
 '박스로',
 '2개',
 '(',
 '10개',
 '##씩',
 '들어있어',
 '20개',
 '##네요',
 ')',
 '주문했습니다',
 '.',
 '의심',
 '##이',
 '많은',
 '타입이라',
 '뭐든',
 '한',
 '번에',
 '주문한',
 '적이',
 '없고',
 ',',
 ',',
 '꼭',
 '먹어보고',
 '써보고',
 '나서야',
 '안심',
 '##하는',
 '타입',
 '##입니다',
 'ㅋ',
 '요즘은',
 '.',
 '.',
 '아카',
 '##시아',
 '##가',
 '많이',
 '없어',
 '.',
 '.',
 '아카',
 '##시아',
 '향',
 '맡아',
 '##보기도',
 '정말',
 '힘든데',
 '.',
 '.',
 '아카',
 '##시아',
 '꿀',
 '##이라니',
 '^',
 '^',
 '너무',
 '반갑',
 '##고',
 '예뻐',
 '주문해',
 '봤습니다',
 '.',
 '일단',
 '너무',
 '작고',
 '예쁘네요',
 '.',
 '더운',
 '여름에',
 '밖에서',
 '운동하는',
 '딸을',
 '위해',
 '건강한',
 '간식으로',
 '챙겨',
 '주려고',
 '합니다',
 '.',
 '예쁜',
 '포장이',
 '##라',
 '예민한',
 '아이',
 '마음에',
 '##도',
 '쏙',
 '드는',
 '듯합니다',
 '.',
 '배송도',
 '빠르고',
 '좋은',
 '제품',
 '좋은',
 '가격에',
 '잘',
 '산',
 '걸',
 '같아',
 '기분',
 '좋네요',
 '.',
 '^',
 '^']